In [1]:
import pandas as pd
from dateutil.parser import parse

In [2]:
invoices = pd.read_pickle("invoices_new.pkl")
#invoices

In [3]:
with open("expired_invoices.txt",'r') as file:
    expired_invoice_id = file.read().split(',')

#expired_invoice_id

In [4]:
#Converting Expired Invoice Id-s to int type
expired_invoice_id = [int(id.strip()) for id in expired_invoice_id]

In [5]:
print("The first five expired invoice id-s:")
print(expired_invoice_id[:5])

The first five expired invoice id-s:
[305869, 377307, 391273, 385290, 331902]


In [6]:
print("Type of Invoice data:", type(invoices))
print("Num of the invoices:", len(invoices))

Type of Invoice data: <class 'list'>
Num of the invoices: 100


In [8]:
def clean_data(invoices):
    cleaned_invoices = []
    for invoice in invoices:
        if 'id' not in invoice or 'created_on' not in invoice or 'items' not in invoice:
            continue
        
        #invoice_id to int type
        invoice_id = invoice.get('id')
        if isinstance(invoice_id, str):
            try:
                invoice_id = int(invoice_id)
            except ValueError:
                continue
        
        #parsing and validating date
        try:
            created_on = parse(invoice.get('created_on'))
            if created_on.day > 28:  #simple way to fix invalid dates
                continue
        except ValueError:
            continue
        
        #validating and cleaning items
        cleaned_items = []
        for item in invoice['items']:
            try:
                item_id = int(item['item']['id'])
                unit_price = float(item['item']['unit_price'])
                quantity = float(item['quantity'])
                item['item']['id'] = item_id
                item['item']['unit_price'] = unit_price
                item['quantity'] = quantity
                cleaned_items.append(item)
            except (ValueError, TypeError, KeyError):
                continue
        
        #kepping invoices with valid items
        if cleaned_items:
            invoice['id'] = invoice_id
            invoice['created_on'] = created_on
            invoice['items'] = cleaned_items
            cleaned_invoices.append(invoice)
    
    return cleaned_invoices

In [10]:
class DataExtractor:
    def __init__(self, invoice_file_path, expired_ids_file_path):
        self.invoice_file_path = invoice_file_path
        self.expired_ids_file_path = expired_ids_file_path
        self.invoices = None
        self.expired_invoice_ids = None
        self.data = None
    
    def load_data(self):
        self.invoices = pd.read_pickle(self.invoice_file_path)
        self.invoices = clean_data(self.invoices)
        
        with open(self.expired_ids_file_path, 'r') as file:
            self.expired_invoice_ids = [int(id.strip()) for id in file.read().split(',')]
    
    def transform_data(self):
        invoice_records = []
        type_conversion = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}
        
        for invoice in self.invoices:
            invoice_id = invoice.get('id')
            created_on = pd.to_datetime(invoice.get('created_on'), errors='coerce')
            
            total_invoice_price = sum(item['item']['unit_price'] * item['quantity'] for item in invoice['items'])
            
            is_expired = invoice_id in self.expired_invoice_ids
            
            for item in invoice['items']:
                item_id = item['item']['id']
                item_name = item['item']['name']
                item_type = type_conversion.get(item['item']['type'], 'Other')
                unit_price = item['item']['unit_price']
                quantity = item['quantity']
                total_price = unit_price * quantity
                percentage_in_invoice = total_price / total_invoice_price if total_invoice_price > 0 else 0

                invoice_records.append({
                    'invoice_id': invoice_id,
                    'created_on': created_on,
                    'invoiceitem_id': item_id,
                    'invoiceitem_name': item_name,
                    'type': item_type,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'percentage_in_invoice': percentage_in_invoice,
                    'is_expired': is_expired
                })
        
        self.data = pd.DataFrame(invoice_records)
        self.data = self.data.astype({
            'invoice_id': int,
            'created_on': 'datetime64[ns]',
            'invoiceitem_id': int,
            'invoiceitem_name': str,
            'type': str,
            'unit_price': int,
            'total_price': int,
            'percentage_in_invoice': float,
            'is_expired': bool
        })
        self.data.sort_values(by=['invoice_id', 'invoiceitem_id'], inplace=True)
    
    def get_transformed_data(self):
        return self.data

In [11]:
invoice_path = "invoices_new.pkl"
expired_ids_path = "expired_invoices.txt"

In [12]:
extractor = DataExtractor(invoice_path, expired_ids_path)
extractor.load_data()
extractor.transform_data()
transformed_data = extractor.get_transformed_data()

In [13]:
#transformed invoices into a csv file
transformed_data.to_csv("transformed_invoices.csv", index=False)

print(transformed_data.head())

     invoice_id created_on  invoiceitem_id invoiceitem_name       type  \
259      301695 2019-04-26          103215        ii_103215      Other   
257      301695 2019-04-26          166227        ii_166227  Equipment   
260      301695 2019-04-26          171394        ii_171394      Other   
258      301695 2019-04-26          195625        ii_195625      Other   
104      304245 2019-03-17          121446        ii_121446    Service   

     unit_price  total_price  percentage_in_invoice  is_expired  
259         135          945               0.374851       False  
257         118          708               0.280841       False  
260         128          128               0.050774       False  
258         148          740               0.293534       False  
104         158          632               0.128429       False  
